### ResNet50 Keras baseline model

This notebook takes you through some important steps in building a deep convnet in Keras for multilabel classification of brain CT scans. 

In [ ]:
import numpy as np
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import collections
from tqdm import tqdm_notebook as tqdm
from datetime import datetime

from math import ceil, floor
import cv2

import tensorflow as tf
import keras

import sys

from keras_applications.resnet import ResNet50

from sklearn.model_selection import ShuffleSplit

### 1. Helper functions

For windowing the input images (thanks to fellow competitors' notebooks\*), and to normalize the pixel values between -1 and 1.

\* Source: https://www.kaggle.com/omission/eda-view-dicom-images-with-correct-windowing

In [ ]:
def _get_first_of_dicom_field_as_int(x):
    if type(x) == pydicom.multival.MultiValue:
        return int(x[0])
    else:
        return int(x)

def _get_windowing(data):
    dicom_fields = [data.WindowCenter, data.WindowWidth, data.RescaleSlope, data.RescaleIntercept]
    return [_get_first_of_dicom_field_as_int(x) for x in dicom_fields]


def _window_image(img, window_center, window_width, slope, intercept):
    img = (img * slope + intercept)
    img_min = window_center - window_width//2
    img_max = window_center + window_width//2
    img[img<img_min] = img_min
    img[img>img_max] = img_max
    return img 

def _normalize(img):
    if img.max() == img.min():
        return np.zeros(img.shape)
    return 2 * (img - img.min())/(img.max() - img.min()) - 1


def _read(path, desired_size=(224, 224)):
    """Will be used in DataGenerator"""
    
    dcm = pydicom.dcmread(path)

    window_params = _get_windowing(dcm) # (center, width, slope, intercept)

    try:
        # dcm.pixel_array might be corrupt (one case so far)
        img = _window_image(dcm.pixel_array, *window_params)
    except:
        img = np.zeros(desired_size)

    img = _normalize(img)

    if desired_size != (512, 512):
        # resize image
        img = cv2.resize(img, desired_size, interpolation=cv2.INTER_LINEAR)

    return img[:,:,np.newaxis]


### 2. Data generators

You could make this in to just one DataGenerator, but I kept it as two separate DataGenerators (one for training and one for predicting). It inherits from keras.utils.Sequence object and thus should be safe for multiprocessing.


In [ ]:
class TrainDataGenerator(keras.utils.Sequence):

    def __init__(self, list_IDs, labels, batch_size=1, img_size=(512, 512), 
                 img_dir='../input/data/stage_1_train_images/', *args, **kwargs):

        self.list_IDs = list_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir
        self.on_epoch_end()

    def __len__(self):
        return int(ceil(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indices]
        X, Y = self.__data_generation(list_IDs_temp)
        return X, Y

    def on_epoch_end(self):
        self.indices = np.arange(len(self.list_IDs))
        np.random.shuffle(self.indices)

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.img_size, 1))
        Y = np.empty((self.batch_size, 6), dtype=np.float32)
        
        for i, ID in enumerate(list_IDs_temp):
            X[i,] = _read(self.img_dir+ID+".dcm", self.img_size)
            Y[i,] = self.labels.loc[ID].values
        
        return X, Y
    
    
class TestDataGenerator(keras.utils.Sequence):

    def __init__(self, list_IDs, labels, batch_size=1, img_size=(512, 512), 
                 img_dir='../input/data/stage_1_test_images/', *args, **kwargs):

        self.list_IDs = list_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir
        self.on_epoch_end()

    def __len__(self):
        return int(ceil(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indices]
        X = self.__data_generation(list_IDs_temp)
        return X

    def on_epoch_end(self):
        self.indices = np.arange(len(self.list_IDs))

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.img_size, 1))
        
        for i, ID in enumerate(list_IDs_temp):
            X[i,] = _read(self.img_dir+ID+".dcm", self.img_size)
        
        return X

### 3. Model

Basically a combination of three models, which are sequentially concatenated. <br> 

* The initial layer, which will transform/map input image of shape (\_, \_, 1) to another "image" of shape (\_, \_, 3).

* The new input image is then passed through ResNet50 (which I named "engine"). ResNet50 could be replaced by any of the available architectures in keras_application.

* Finally, the output from ResNet50 goes through average pooling followed by a dense output layer.

In [ ]:
def _initial_layer(input_dims):
    inputs = keras.layers.Input(input_dims)
    
    x = keras.layers.Conv2D(filters=3, kernel_size=(1, 1), strides=(1, 1), name="initial_conv2d")(inputs)
    x = keras.layers.BatchNormalization(axis=3, epsilon=1.001e-5, name='initial_bn')(x)
    x = keras.layers.Activation('relu', name='initial_relu')(x)
    
    return keras.models.Model(inputs, x)

class MyDeepModel:
    
    def __init__(self, engine, input_dims, batch_size=5, learning_rate=1e-3, 
                 decay_rate=1.0, decay_steps=1, weights="imagenet", verbose=1):
        
        self.engine = engine
        self.input_dims = input_dims
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.decay_rate = decay_rate
        self.decay_steps = decay_steps
        self.weights = weights
        self.verbose = verbose
        self._build()

    def _build(self):
        
        initial_layer = _initial_layer((*self.input_dims, 1))
    
        engine = self.engine(include_top=False, weights=self.weights, input_shape=(*self.input_dims, 3),
                             backend = keras.backend, layers = keras.layers,
                             models = keras.models, utils = keras.utils)

        x = engine(initial_layer.output)

        x = keras.layers.GlobalAveragePooling2D(name='avg_pool')(x)

        out = keras.layers.Dense(6, activation="sigmoid", name='dense_output')(x)

        self.model = keras.models.Model(inputs=initial_layer.input, outputs=out)

        self.model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(0.0))
    
    
    def fit(self, df, train_idx, img_dir, global_epoch):
        self.model.fit_generator(
            TrainDataGenerator(
                df.iloc[train_idx].index, 
                df.iloc[train_idx], 
                self.batch_size, 
                self.input_dims, 
                img_dir
            ),
            verbose=self.verbose,
            use_multiprocessing=True,
            workers=4,
            callbacks=[
                keras.callbacks.LearningRateScheduler(
                    lambda epoch: self.learning_rate * pow(self.decay_rate, floor(global_epoch / self.decay_steps))
                )
            ]
        )
    
    def predict(self, df, test_idx, img_dir):
        predictions = \
          self.model.predict_generator(
            TestDataGenerator(
                df.iloc[test_idx].index, 
                None, 
                self.batch_size, 
                self.input_dims, 
                img_dir
            ),
            verbose=1,
            use_multiprocessing=True,
            workers=4
        )

        return predictions[:df.iloc[test_idx].shape[0]]
    
    def save(self, path):
        self.model.save_weights(path)
    
    def load(self, path):
        self.model.load_weights(path)

### 4. Read csv files


In [ ]:
def read_testset(filename="../input/rsna-intracranial-hemorrhage-detection/stage_1_sample_submission.csv"):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)
    
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    
    return df

def read_trainset(filename="../input/rsna-intracranial-hemorrhage-detection/stage_1_train.csv"):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)
    
    duplicates_to_remove = [
        1598538, 1598539, 1598540, 1598541, 1598542, 1598543,
        312468,  312469,  312470,  312471,  312472,  312473,
        2708700, 2708701, 2708702, 2708703, 2708704, 2708705,
        3032994, 3032995, 3032996, 3032997, 3032998, 3032999
    ]
    
    df = df.drop(index=duplicates_to_remove)
    df = df.reset_index(drop=True)
    
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    
    return df

    
test_df = read_testset()
df = read_trainset()

In [ ]:
df.head(3)

In [ ]:
test_df.head(3)

### 5. Train model and predict

*Using train, validation and test set* <br>

Training for 3 epochs with Adam optimizer. The learning rate starts at 0.001, with a slight decay (0.75) each epoch. The predictions are then \[exponentially weighted\] averaged over all 3 epochs. Same goes for the test set submission later.

In [ ]:
_TEST_IMAGES = '../input/rsna-intracranial-hemorrhage-detection/stage_1_test_images/'
_TRAIN_IMAGES = '../input/rsna-intracranial-hemorrhage-detection/stage_1_train_images/'

def run(model, df, train_idx, valid_idx, test_df, epochs):
    
    valid_predictions = []
    test_predictions = []
    for global_epoch in range(epochs):

        model.fit(df, train_idx, _TRAIN_IMAGES, global_epoch)
        
        test_predictions.append(model.predict(test_df, range(test_df.shape[0]), _TEST_IMAGES))
        valid_predictions.append(model.predict(df, valid_idx, _TRAIN_IMAGES))
        
        print("validation loss: %.4f" %
              weighted_loss_metric(df.iloc[valid_idx].values, 
                                   np.average(valid_predictions, axis=0, 
                                              weights=[2**i for i in range(len(valid_predictions))]))
             )
    
    return test_predictions, valid_predictions



def weighted_loss_metric(trues, preds, weights=[0.2, 0.1, 0.1, 0.1, 0.1, 0.1], clip_value=1e-7):
    """this is probably not correct, but works OK. Feel free to give feedback."""
    preds = np.clip(preds, clip_value, 1-clip_value)
    loss_subtypes = trues * np.log(preds) + (1 - trues) * np.log(1 - preds)
    loss_weighted = np.average(loss_subtypes, axis=1, weights=weights)
    return - loss_weighted.mean()

# train set (90%) and validation set (10%)
ss = ShuffleSplit(n_splits=5, test_size=0.1, random_state=42).split(df.index)

# will just do one fold
train_idx, valid_idx = next(ss)

# obtain model
model = MyDeepModel(engine=ResNet50, input_dims=(224, 224), batch_size=32, learning_rate=1e-3, 
                    decay_rate=0.75, decay_steps=1, weights="imagenet", verbose=2)

# run 3 epochs and obtain test + validation predictions
test_preds, _ = run(model, df, train_idx, valid_idx, test_df, 3)


### 6. Submit test predictions

In [ ]:
test_df.iloc[:, :] = np.average(test_preds, axis=0, weights=[2**i for i in range(len(test_preds))])

test_df = test_df.stack().reset_index()

test_df.insert(loc=0, column='ID', value=test_df['Image'].astype(str) + "_" + test_df['Diagnosis'])

test_df = test_df.drop(["Image", "Diagnosis"], axis=1)

test_df.to_csv('submission.csv', index=False)

### 7. Improvements

Some improvements that could be made:<br>
* Image augmentation (which can be put in `_read()`)
* Different learning rate and learning rate schedule
* Increased input size
* Train longer (or shorter? :O)
* Add regularization (e.g. `keras.layers.Dropout()` before the output layer)
* Do something about `_initial_layer()`?
<br>
<br>
*Feel free to comment!*
